In [22]:
import json_tricks
import torch
import numpy as np
import matplotlib.pyplot as plt
import random

# This is important for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

answer = {}

# Preparation

In this notebook we will be working on toy problem of reconstructing the hidden 
dependency. We will work with synthetic dataset. As a basic example, we will
start with data

$$f(x) = \sin(x) + \epsilon,$$

where $\epsilon$ is random noise that will simulate measurement inaccuracies.

Our task will be to prepare the network $Net(x)$ that will accept single value $x$
as an argument and yield single value $y$ as a forecast.

Ideally the network should be as close to the hidden dependency as possible:

$$Net(x) \rightarrow \sin(x)$$

We will create all the modules that are necessary to train a network:
- network (2 layer FCNN)
- loss function (MSE)
- optimizer (AdamW)

We will also create a traingin cycle that consists of:
- forward step
- loss calculation
- backpropagation
- optimizer step

# Generating the dataset

First we will generate the clean samples

In [23]:
import torch
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

x_train = torch.rand(100)
x_train = x_train * 20.0 - 10.0

y_train = torch.sin(x_train)

plt.plot(x_train.numpy(), y_train.numpy(), 'o')
plt.title('$y = sin(x)$');

Secondly, generate the noise that will be added to the training data

In [24]:
noise = torch.randn(y_train.shape) / 10.

plt.plot(x_train.numpy(), noise.numpy(), 'o')
plt.axis([-10, 10, -1, 1])
plt.title('Gaussian noise');

Lastly, add the noise to the clean data and get the spoiled data that we will use for training

In [25]:
y_train = y_train + noise
plt.plot(x_train.numpy(), y_train.numpy(), 'o')
plt.title('noisy sin(x)')
plt.xlabel('x_train')
plt.ylabel('y_train');

In [26]:
x_train.unsqueeze_(1);
y_train.unsqueeze_(1);

# Validation dataset

For validation we will use only clean equidistant data. Thus we will not generate noise for this set.

In [27]:
x_validation = torch.linspace(-10, 10, 100)
y_validation = torch.sin(x_validation.data)
plt.plot(x_validation.numpy(), y_validation.numpy(), 'o')
plt.title('sin(x)')
plt.xlabel('x_validation')
plt.ylabel('y_validation');

print(x_validation.dtype)
print(x_validation)

In [7]:
x_validation.unsqueeze_(1)
y_validation.unsqueeze_(1);

# Model construction

Fill out the class for 2-layer Feed Forward Neural Network with Sigmoid hidden layer.

Class should consist of two methods:
* `__init__` -- constructor, where the layers should be defined:
    * `Linear 1 -> n_hidden_neurons`
    * `sigmoid` activation
    * `Linear n_hidden_neurons -> 1`
* `forward` -- pass the signal through the network:
    * `x -> linear 1 -> sigmoid -> linear 2`

In [8]:
class SineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        ...
        ## YOUR CODE HERE ##

    def forward(self, x):
        ...
        ## YOUR CODE HERE ##
        return x

sine_net = SineNet(20)

# Prediction

Here we will make prediction using our neural network. The main method here is `forward` that we have programmed within our `torch.Module` class.

Below we will plot out predictions to understand how they are related to the validation data.

In [9]:
y_pred = sine_net.forward(x_validation)
answer['init_pred'] = y_pred.detach().numpy()

def predict(net, x, y):
    y_pred = net.forward(x)

    plt.plot(x.numpy(), y.numpy(), 'o', label='Groud truth')
    plt.plot(x.numpy(), y_pred.data.numpy(), 'o', c='r', label='Prediction');
    plt.legend(loc='upper left')
    plt.xlabel('$x$')
    plt.ylabel('$y$')

predict(sine_net, x_validation, y_validation)

We see that the untrained neural network predict something, maybe even some dependency, but it is not related to the dependency that we want to reconstruct. 

Now let us train the network (tune the parameters to minimize the misfit between the labels and predictions).

# Loss function

Now we will define the function using which we will measure the misfit.

The requirements for this function are:
* the lower the loss is the better the predictions are
* slope should be non-zero for the majority of the locations

In our case it will be mean squared error:
$$\frac{1}{N} \sum_{i=1}^N \left(Net(x_i) - t_i\right)^2$$

The lower this deviation is, the closer the prediction of the Neural Network to target.

Code this loss function. Note that `pred` and `target` are the vectors.

In [10]:
def loss(pred, target):
    ...
    ## YOUR CODE HERE ##

answer['loss'] = [loss(torch.tensor([v]), torch.tensor([0])).detach().numpy() for v in np.linspace(-10, 10, 100)]

# Optimizer

Now we have to tune the parameters minimising the loss function. Select `AdamW` optimizer from `torch.optim`.

As a first argument you should pass which parameters the optimizer should take care of. One can access whole set of parameters of `torch.Module` using `parameters()` method.

Set `lr` (learning rate) to `1.0e-2`.

In [11]:
## YOUR CODE HERE ##


# Training procedure

Now everything is ready for training.

Compound the training cycle out of the following step:
1. reset the gradients for trainable weights (`zero_grad()` method of the optimizer)
2. forward signal propagation through the network (take vector of input signals `x_train` and call `forward` method of the `torch.nn.Module` with that `x_train` as an argument)
3. loss value comutation (take`loss` that we have created above and comute misfit of prediction and target labels `y_train`)
4. backpropagation through the network, computation of loss derivatives through every of the weights of the network (`backward` method of `torch.nn.Module`)
5. optimizer step as all of the gradients for every of the weights are known (`step` method of the `Optimizer`)

Repeat the steps above for some amount of times. Say, for `1000` and observe the change of the network predictions

In [12]:
from tqdm import trange
for epoch_index in trange(2000):
    ...
    ## YOUR CODE HERE ##

y_pred = sine_net.forward(x_validation)
answer['trained_pred'] = y_pred.detach().numpy()

predict(sine_net, x_validation, y_validation)

In [19]:
import lovely_tensors as lt
lt.monkey_patch()

print(x_train.dtype)
print(y_train.dtype)
print(x_train)
print(y_train)
print(x_validation)


# Conclusion

In this notebook we have covered all of the steps that the coder encounters while programming training procedure of a Neural Network.

In serious projects every part of this process contatins tonns of code, but the philosophy everywhere is the same:

* Get data
* Define the Network
* Define Loss
* Select optimizer
* Run training cycle

In [13]:
json_tricks.dump(answer, open('.answer.json', 'w'))